### Testing the extraction parameters

In [51]:
using PyCall
using Statistics , StatsBase
using  DataFrames, Printf

import PyPlot
sns= pyimport("seaborn")

using DelimitedFiles

## directory
rootdir = ENV["GAIA_ROOT"]
wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable"
plotdir = "$rootdir/products/testfull2"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering


cd(wdir)

In [52]:
mutable struct _extractParam
    rootdir::String
    wdir::String
    votdir::String
    plotdir::String
    ocdir::String

    minQ::Float64
    minstars::Int
    maxstars::Int
    forcedminstars::Int
    mingoodsolution::Int
    niterqminq::Int
    nburnout::Int
    niter::Int

    epsmean::Float64
    epsdisp::Float64
    min_nei::Int
    min_cl::Int
    ncoredisp::Int
    w3dmean::Float64
    w3ddisp::Float64
    wvelmean::Float64
    wveldisp::Float64
    whrdmean::Float64
    whrddisp::Float64

    aperture2d::Float64
    maxaperture2d::Float64 
    aperture3d::Float64    
    maxaperture3d::Float64
    aperturev::Float64   
    maxaperturev::Float64
    nboot::Int
    labels::String
    
    _extractParam()= new()
end

## return the default parameters
##
function _set_default_params()::_extractParam
    def = _extractParam()
    
    ## Directories
    def.rootdir =  ENV["GAIA_ROOT"]
    def.wdir    = "$rootdir/e2e_products"
    def.votdir  = "$wdir/votable"
    def.plotdir = "$wdir/plots-select"
    def.ocdir   = "$wdir/ocfull"

    ## MCMC
    ##
    def.minQ    = 2.7
    def.minstars = 40
    def.maxstars = 10000
    def.forcedminstars = 30
    def.mingoodsolution = 10        # minimum good solution for Q in check_qminqstar
    def.niterqminq = 500            # check_qminqstar..

    def.nburnout  = 2000            # burn-in iterations
    def.niter     = 15000           # MCMC iterations

    ## prior settings
    ##
    def.epsmean   = 2.5             # epsilon mean
    def.epsdisp   = 1.5             # epsilon dispersion
    def.min_nei   = 10              # min_nei mean
    def.min_cl    = 15              # min_cl mean
    def.ncoredisp = 10              # dispersion for min_nei and min_cl
    def.w3dmean   = 6.0
    def.w3ddisp   = 4.0
    def.wvelmean  = 6.0
    def.wveldisp  = 4.0
    def.whrdmean  = 2.0
    def.whrddisp  = 1.5

    ## Metrics find_clusters
    def.aperture2d    = 1.5
    def.maxaperture2d = 15
    def.aperture3d    = 3
    def.maxaperture3d = 20
    def.aperturev     = 3.0
    def.maxaperturev  = 20
    def.nboot         = 50          # bootstrap number

    def.labels        = "Qc"        # method to get labels: Qc|Qn
    
    return(def)
end

_set_default_params (generic function with 1 method)

In [53]:
function _read_params(file)
    default= _set_default_params()
    t= readdlm(file,comments=true, comment_char='#')
    
    
end

function _set_params!(def, parstr,value)
    if parstr == "rootdir" def.rootdir= value end
    if parstr == "wdir"    def.wdir= value end
    if parstr == "votdir"  def.votdir= value end
    if parstr == "plotdir" def.plotdir= value end
    if parstr == "ocdir"   def.ocdir= value end

    if parstr == "minQ"            def.minQ= value end
    if parstr == "minstars"        def.minstars= value end
    if parstr == "maxstars"        def.maxstars= value end
    if parstr == "forcedminstars"  def.forcedminstars= value end
    if parstr == "mingoodsolution" def.mingoodsolutions= value end
    if parstr == "niterqminq"      def.niterqminq= value end

    
    def.nburnout  = 2000            # burn-in iterations
    def.niter     = 15000           # MCMC iterations

    ## prior settings
    ##
    def.epsmean   = 2.5             # epsilon mean
    def.epsdisp   = 1.5             # epsilon dispersion
    def.min_nei   = 10              # min_nei mean
    def.min_cl    = 15              # min_cl mean
    def.ncoredisp = 10              # dispersion for min_nei and min_cl
    def.w3dmean   = 6.0
    def.w3ddisp   = 4.0
    def.wvelmean  = 6.0
    def.wveldisp  = 4.0
    def.whrdmean  = 2.0
    def.whrddisp  = 1.5

    ## Metrics find_clusters
    def.aperture2d    = 1.5
    def.maxaperture2d = 15
    def.aperture3d    = 3
    def.maxaperture3d = 20
    def.aperturev     = 3.0
    def.maxaperturev  = 20
    def.nboot         = 50          # bootstrap number

    def.labels        = "Q
end

_read_params (generic function with 1 method)

In [54]:
## Main
function main(file)
    _read_params(file)
        
end

main (generic function with 1 method)

In [55]:
file= "default.ext"
main(file)

26×3 Array{Any,2}:
 "minQ"             "="      2.7
 "minstars"         "="     40
 "forcedminstars"   "="     30
 "mingoodsolution"  "="     10
 "niterqminq"       "="    500
 "nburnout"         "="   2000
 "niter"            "="  15000
 "epsmean"          "="      2.5
 "epsdisp"          "="      1.5
 "min_nei"          "="     10
 "min_cl"           "="     15
 "ncoredisp"        "="     10
 "w3dmean"          "="      6.0
 "w3ddisp"          "="      4.0
 "wvelmean"         "="      6.0
 "wveldisp"         "="      4.0
 "whrdmean"         "="      2.0
 "whrddisp"         "="      1.5
 "aperture2d"       "="      1.5
 "maxaperture2d"    "="     15.0
 "aperture3d"       "="      3.0
 "maxaperture3d"    "="     20
 "aperturev"        "="      3.0
 "maxaperturev"     "="     20.0
 "nboot"            "="     50.0
 "labels"           "="       "Qc"